In [1]:
#@title Installs
#!pip install --upgrade tensorflow-model-remediation
#!pip install --upgrade fairness-indicators

In [2]:
#@title Imports
import copy
import os
import requests
import tempfile

from fairness_indicators.examples import util
import model_remediation.min_diff as md
from google.protobuf import text_format
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_model_analysis as tfma
import tensorflow_data_validation as tfdv
from tensorflow_model_analysis.addons.fairness.post_export_metrics import fairness_indicators
from tensorflow_model_analysis.addons.fairness.view import widget_view

In [3]:
#@title Preprocess Data
# We use a helper utility to preprocessed data for convenience and speed.
data_train, data_validate, validate_tfrecord_file, labels_train, labels_validate = util.download_and_process_civil_comments_data()

In [4]:
#@title Constants
TEXT_FEATURE = 'comment_text'
LABEL = 'toxicity'
BATCH_SIZE = 128

In [5]:
#@title Seeds
np.random.seed(1)
tf.random.set_seed(1)

In [ ]:
#@title Train Model

use_pretrained_model = True #@param {type:"boolean"}

if use_pretrained_model:
  URL = 'https://storage.googleapis.com/civil_comments_model/baseline_model.h5'
  FILEPATH = 'baseline_pretrained_model.h5'
  r = requests.get(URL, allow_redirects=True)
  open(FILEPATH, 'wb').write(r.content)
  baseline_model = tf.keras.models.load_model(FILEPATH, custom_objects={'KerasLayer' : hub.KerasLayer})

else:
  optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
  loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)

  baseline_model = util.create_keras_sequential_model()
  
  baseline_model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

  baseline_model.fit(x=data_train[TEXT_FEATURE],
                     y=labels_train, batch_size=BATCH_SIZE,
                     epochs=10)


In [ ]:
#@title Save Model
base_dir = tempfile.mkdtemp(prefix='saved_models')
baseline_model_location = os.path.join(base_dir, 'model_export_baseline')
baseline_model.save(baseline_model_location, save_format='tf')

In [8]:
#@title Run Model Analysis

# We use a helper utility to hide the evaluation logic for readability.
base_dir = tempfile.mkdtemp(prefix='eval') 
eval_subdir = 'eval_results_baseline'
eval_result = util.get_eval_results(baseline_model_location, base_dir,
                                    eval_subdir, validate_tfrecord_file)

In [9]:
#@title Render Evaluation Results

widget_view.render_fairness_indicator(eval_result)

In [10]:
#@title Create Mindiff DataFrames

# Create masks for the sensitive and nonsensitive groups
minority_mask = data_train.religion.apply(
    lambda x: any(religion in x for religion in 
                  ('atheist', 'jewish', 'muslim')))
majority_mask = data_train.religion.apply(lambda x: x == "['christian']")

# Select nontoxic examples, so Mindiff will be able to reduce sensitive FP rate.
true_negative_mask = data_train['toxicity'] == 0

data_train_main = copy.copy(data_train)
data_train_sensitive = data_train[minority_mask & true_negative_mask]
data_train_nonsensitive = data_train[majority_mask & true_negative_mask]

In [11]:
#@title Create Mindiff Datasets

# Convert the pandas DataFrames to Datasets.
dataset_train_main = tf.data.Dataset.from_tensor_slices(
    (data_train_main['comment_text'].values, 
     data_train_main.pop(LABEL).values.reshape(-1,1) * 1.0)).batch(BATCH_SIZE)
dataset_train_sensitive = tf.data.Dataset.from_tensor_slices(
    (data_train_sensitive['comment_text'].values, 
     data_train_sensitive.pop(LABEL).values.reshape(-1,1) * 1.0)).batch(BATCH_SIZE)
dataset_train_nonsensitive = tf.data.Dataset.from_tensor_slices(
    (data_train_nonsensitive['comment_text'].values, 
     data_train_nonsensitive.pop(LABEL).values.reshape(-1,1) * 1.0)).batch(BATCH_SIZE)

In [ ]:
#@title Train Model

use_pretrained_model = True #@param {type:"boolean"}

base_dir = tempfile.mkdtemp(prefix='saved_models')
min_diff_model_location = os.path.join(base_dir, 'model_export_min_diff')

if use_pretrained_model:
  URL = 'https://storage.googleapis.com/civil_comments_model/min_diff_model.h5'
  FILEPATH = 'min_diff_pretrained_model.h5'
  r = requests.get(URL, allow_redirects=True)
  open(FILEPATH, 'wb').write(r.content)
  min_diff_model = tf.keras.models.load_model(FILEPATH, custom_objects={'KerasLayer' : hub.KerasLayer})
  
  min_diff_model.save(min_diff_model_location, save_format='tf')

else:
  # Create the dataset that will be passed to the MinDiffModel during training.
  dataset = md.keras.utils.input_utils.pack_min_diff_data(
      dataset_train_main, dataset_train_sensitive, dataset_train_nonsensitive)

  # Create the original model.
  original_model = util.create_keras_sequential_model()
  
  # Wrap the original model in a MinDiffModel, passing in one of the min diff
  # losses and using a moderately high strength.
  min_diff_strength = 1.5
  min_diff_loss = md.losses.MMDLoss('gauss') # TODO:  try AbsCorrLoss()
  min_diff_model = md.keras.MinDiffModel(original_model, min_diff_loss, min_diff_strength)

  # Compile the model normally after wrapping the original model.  Note that
  # this means we use the baseline's model's loss here.
  optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
  loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)
  min_diff_model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

  min_diff_model.fit(dataset, epochs=10)

  min_diff_model.save_original_model(min_diff_model_location, save_format='tf')

In [13]:
#@title Run Model Analysis
min_diff_eval_subdir = 'eval_results_min_diff'
min_diff_eval_result = util.get_eval_results(min_diff_model_location, base_dir,
                                    min_diff_eval_subdir, validate_tfrecord_file)


In [14]:
#@title Render Evaluation Results

widget_view.render_fairness_indicator(min_diff_eval_result)